**Task 2**

In [2]:
import seaborn as sns
import sqlite3
import pandas as pd

# Load the penguins dataset
penguins = sns.load_dataset("penguins").dropna()
penguins.info()


<class 'pandas.core.frame.DataFrame'>
Index: 333 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            333 non-null    object 
 1   island             333 non-null    object 
 2   bill_length_mm     333 non-null    float64
 3   bill_depth_mm      333 non-null    float64
 4   flipper_length_mm  333 non-null    float64
 5   body_mass_g        333 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 20.8+ KB


In [3]:
# Create a unique numeric ID for each island, we go through each unique name of the island in the dataset and assign a unique number to this island name
island_mapping = {name: idx+1 for idx, name in enumerate(penguins['island'].unique())}
penguins['island_id'] = penguins['island'].map(island_mapping)

# We do the same with penguins, we add a unique animal ID for each penguin 
penguins['animal_id'] = range(1, len(penguins) + 1)

# Connect to SQLite database
conn = sqlite3.connect('data/penguins.db')
cursor = conn.cursor()

# Enable foreign key constraints
cursor.execute("PRAGMA foreign_keys = ON;")

# Create `Islands` table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Islands (
    island_id INTEGER PRIMARY KEY,
    name TEXT UNIQUE
)
''')

# Create `Penguins` table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Penguins (
    animal_id INTEGER PRIMARY KEY AUTOINCREMENT,
    species TEXT,
    bill_length_mm REAL,
    bill_depth_mm REAL,
    flipper_length_mm REAL,
    body_mass_g REAL,
    sex TEXT,
    island_id INTEGER,
    FOREIGN KEY (island_id) REFERENCES Islands(island_id)
)
''')

# Insert island data into `Islands` table
islands_df = pd.DataFrame(list(island_mapping.items()), columns=['name', 'island_id'])
islands_df.to_sql('Islands', conn, if_exists='replace', index=False)

# Insert penguin data into `Penguins` table
penguins_selected = penguins[['animal_id', 'species', 'bill_length_mm', 'bill_depth_mm',
                              'flipper_length_mm', 'body_mass_g', 'sex', 'island_id']]
penguins_selected.to_sql('Penguins', conn, if_exists='replace', index=False)

# Verify tables
print(pd.read_sql("SELECT * FROM Islands;", conn))
print(pd.read_sql("SELECT * FROM Penguins LIMIT 5;", conn))



        name  island_id
0  Torgersen          1
1     Biscoe          2
2      Dream          3
   animal_id species  bill_length_mm  bill_depth_mm  flipper_length_mm  \
0          1  Adelie            39.1           18.7              181.0   
1          2  Adelie            39.5           17.4              186.0   
2          3  Adelie            40.3           18.0              195.0   
3          4  Adelie            36.7           19.3              193.0   
4          5  Adelie            39.3           20.6              190.0   

   body_mass_g     sex  island_id  
0       3750.0    Male          1  
1       3800.0  Female          1  
2       3250.0  Female          1  
3       3450.0  Female          1  
4       3650.0    Male          1  


In [4]:
# Close connection
conn.close()